# Meridian City Hospital - East ER Efficiency Analysis

**Comprehensive Analysis for ER Datathon**

This notebook performs a complete end-to-end analysis of the Emergency Room efficiency at Meridian City Hospital's East Location. The analysis includes:

1. Data Loading & Cleaning
2. Feature Engineering
3. Exploratory Data Analysis (EDA)
4. Machine Learning Models (Wait Time, Satisfaction, 15-min Target, Disposition)
5. Statistical Correlations
6. Scenario Simulations
7. Executive Summary & Recommendations

## 1. Setup & Imports

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import os

# Machine Learning Libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    classification_report, confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score
)

# Visualization settings
%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
warnings.filterwarnings('ignore')

# Create outputs directory
os.makedirs('outputs', exist_ok=True)

print("✓ All libraries imported successfully")
print("✓ Outputs directory ready")

## 2. Data Loading

Loading all required CSV datasets for the East ER Location analysis.

In [ ]:
# Load all datasets
patients_df = pd.read_csv('data/Hospital_Patients.csv')
visits_df = pd.read_csv('data/Hospital_Visits.csv')
staffing_df = pd.read_csv('data/Hospital_Staffing_EAST_LOCATION.csv')
facility_df = pd.read_csv('data/Hospital_Facility.csv')
outcomes_df = pd.read_csv('data/Hospital_Outcomes.csv')

# Print dataset summaries
print("Dataset Record Counts:")
print("=" * 50)
print(f"Patients: {len(patients_df):,} records")
print(f"Visits: {len(visits_df):,} records")
print(f"Staffing: {len(staffing_df):,} records")
print(f"Facility: {len(facility_df):,} records")
print(f"Outcomes: {len(outcomes_df):,} records")

In [ ]:
# Display sample data from each dataset
print("\n" + "=" * 50)
print("PATIENTS DATASET (Sample)")
print("=" * 50)
display(patients_df.head(3))

print("\n" + "=" * 50)
print("VISITS DATASET (Sample)")
print("=" * 50)
display(visits_df.head(3))

print("\n" + "=" * 50)
print("STAFFING DATASET (Sample)")
print("=" * 50)
display(staffing_df.head(3))

print("\n" + "=" * 50)
print("FACILITY DATASET (Sample)")
print("=" * 50)
display(facility_df.head(3))

print("\n" + "=" * 50)
print("OUTCOMES DATASET (Sample)")
print("=" * 50)
display(outcomes_df.head(3))

## 3. Data Cleaning & Preprocessing

Cleaning and standardizing all datasets before analysis.

In [ ]:
def parse_flexible_date(date_str):
    """
    Robust date parser that handles multiple timestamp formats.
    
    Handles formats like:
    - 2024-01-15 14:30:00
    - 01/15/2024 2:30 PM
    - 2024-01-15T14:30:00
    - and more
    """
    if pd.isna(date_str):
        return pd.NaT
    
    # List of common date formats
    formats = [
        '%Y-%m-%d %H:%M:%S',
        '%m/%d/%Y %I:%M %p',
        '%m/%d/%Y %H:%M',
        '%Y-%m-%dT%H:%M:%S',
        '%Y-%m-%d',
        '%m/%d/%Y',
        '%d/%m/%Y %H:%M:%S',
        '%d/%m/%Y %I:%M %p'
    ]
    
    for fmt in formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except:
            continue
    
    # If all formats fail, try pandas default parser
    try:
        return pd.to_datetime(date_str)
    except:
        return pd.NaT

print("✓ Date parser function defined")

In [ ]:
# Clean Visits Dataset
print("Cleaning Visits dataset...")

# Parse all timestamp columns
timestamp_cols = ['Arrival_Time', 'Registration_Time', 'Triage_Start', 'Triage_End', 
                  'Doctor_Assigned_Time', 'Treatment_Start', 'Departure_Time']

for col in timestamp_cols:
    if col in visits_df.columns:
        visits_df[col] = visits_df[col].apply(parse_flexible_date)

# Standardize triage levels
if 'Triage_Level' in visits_df.columns:
    visits_df['Triage_Level'] = visits_df['Triage_Level'].str.strip().str.upper()

# Standardize shift
if 'Shift' in visits_df.columns:
    visits_df['Shift'] = visits_df['Shift'].str.strip().str.title()

# Filter for East ER only
if 'Hospital_ID' in visits_df.columns:
    visits_df = visits_df[visits_df['Hospital_ID'] == 'MC_ER_EAST'].copy()

print(f"✓ Visits dataset cleaned: {len(visits_df):,} East ER records")

In [ ]:
# Clean Outcomes Dataset
print("Cleaning Outcomes dataset...")

# Standardize disposition
if 'Disposition' in outcomes_df.columns:
    outcomes_df['Disposition'] = outcomes_df['Disposition'].str.strip().str.title()

# Filter for East ER
if 'Hospital_ID' in outcomes_df.columns:
    outcomes_df = outcomes_df[outcomes_df['Hospital_ID'] == 'MC_ER_EAST'].copy()

print(f"✓ Outcomes dataset cleaned: {len(outcomes_df):,} records")

In [ ]:
# Clean Patients Dataset
print("Cleaning Patients dataset...")

# No specific cleaning needed for patients, but verify structure
print(f"✓ Patients dataset verified: {len(patients_df):,} records")

# Clean Staffing Dataset
print("Cleaning Staffing dataset...")

# Parse date column if exists
if 'Date' in staffing_df.columns:
    staffing_df['Date'] = staffing_df['Date'].apply(parse_flexible_date)

# Standardize shift
if 'Shift' in staffing_df.columns:
    staffing_df['Shift'] = staffing_df['Shift'].str.strip().str.title()

print(f"✓ Staffing dataset cleaned: {len(staffing_df):,} records")

# Clean Facility Dataset
print("Cleaning Facility dataset...")
print(f"✓ Facility dataset verified: {len(facility_df):,} records")

print("\n" + "=" * 50)
print("ALL DATASETS CLEANED SUCCESSFULLY")
print("=" * 50)

## 4. Feature Engineering

Creating calculated fields for analysis including wait times, temporal features, and staffing ratios.

In [ ]:
# Calculate time intervals (in minutes)
print("Calculating time intervals...")

# Wait to Registration
visits_df['Wait_to_Registration'] = (
    (visits_df['Registration_Time'] - visits_df['Arrival_Time']).dt.total_seconds() / 60
).fillna(0)

# Registration Duration
visits_df['Registration_Duration'] = (
    (visits_df['Triage_Start'] - visits_df['Registration_Time']).dt.total_seconds() / 60
).fillna(0)

# Wait to Triage
visits_df['Wait_to_Triage'] = (
    (visits_df['Triage_Start'] - visits_df['Arrival_Time']).dt.total_seconds() / 60
).fillna(0)

# Triage Duration
visits_df['Triage_Duration'] = (
    (visits_df['Triage_End'] - visits_df['Triage_Start']).dt.total_seconds() / 60
).fillna(0)

# Wait to Doctor
visits_df['Wait_to_Doctor'] = (
    (visits_df['Doctor_Assigned_Time'] - visits_df['Triage_End']).dt.total_seconds() / 60
).fillna(0)

# Treatment Duration
visits_df['Treatment_Duration'] = (
    (visits_df['Departure_Time'] - visits_df['Treatment_Start']).dt.total_seconds() / 60
).fillna(0)

# Total ER Time
visits_df['Total_ER_Time'] = (
    (visits_df['Departure_Time'] - visits_df['Arrival_Time']).dt.total_seconds() / 60
).fillna(0)

# Time to Doctor (from arrival)
visits_df['Time_to_Doctor'] = (
    (visits_df['Doctor_Assigned_Time'] - visits_df['Arrival_Time']).dt.total_seconds() / 60
).fillna(0)

# Create 15-minute target flag
visits_df['Seen_Within_15min'] = (visits_df['Time_to_Doctor'] <= 15).astype(int)

print("✓ Time intervals calculated")

In [ ]:
# Create temporal features
print("Creating temporal features...")

visits_df['Date'] = visits_df['Arrival_Time'].dt.date
visits_df['Hour'] = visits_df['Arrival_Time'].dt.hour
visits_df['Day_of_Week'] = visits_df['Arrival_Time'].dt.day_name()
visits_df['Is_Weekend'] = visits_df['Arrival_Time'].dt.dayofweek.isin([5, 6]).astype(int)

# Assign shift by hour (if not already present)
def assign_shift(hour):
    if 7 <= hour < 15:
        return 'Day'
    elif 15 <= hour < 23:
        return 'Evening'
    else:
        return 'Night'

if 'Shift' not in visits_df.columns or visits_df['Shift'].isna().any():
    visits_df['Shift'] = visits_df['Hour'].apply(assign_shift)

print("✓ Temporal features created")

In [ ]:
# Merge all datasets
print("Merging datasets...")

# Start with visits as the base
merged_df = visits_df.copy()

# Merge with patients
if 'Patient_ID' in merged_df.columns and 'Patient_ID' in patients_df.columns:
    merged_df = merged_df.merge(patients_df, on='Patient_ID', how='left', suffixes=('', '_patient'))
    print(f"  ✓ Merged with patients: {len(merged_df):,} records")

# Merge with outcomes
if 'Visit_ID' in merged_df.columns and 'Visit_ID' in outcomes_df.columns:
    merged_df = merged_df.merge(outcomes_df, on='Visit_ID', how='left', suffixes=('', '_outcome'))
    print(f"  ✓ Merged with outcomes: {len(merged_df):,} records")

# Merge with staffing (by Date and Shift)
if 'Date' in merged_df.columns and 'Shift' in merged_df.columns:
    # Ensure Date columns are same type
    staffing_df['Date'] = pd.to_datetime(staffing_df['Date']).dt.date
    merged_df = merged_df.merge(
        staffing_df[['Date', 'Shift', 'Doctors_on_Duty', 'Nurses_on_Duty', 'Fast_Track_Open']], 
        on=['Date', 'Shift'], 
        how='left'
    )
    print(f"  ✓ Merged with staffing: {len(merged_df):,} records")

print("\n✓ All datasets merged successfully")

In [ ]:
# Calculate staffing ratios
print("Calculating staffing ratios...")

# Count patients per shift/date
if 'Date' in merged_df.columns and 'Shift' in merged_df.columns:
    shift_counts = merged_df.groupby(['Date', 'Shift']).size().reset_index(name='Patients_Count')
    merged_df = merged_df.merge(shift_counts, on=['Date', 'Shift'], how='left')
    
    # Calculate ratios
    if 'Nurses_on_Duty' in merged_df.columns:
        merged_df['Patients_per_Nurse'] = (
            merged_df['Patients_Count'] / merged_df['Nurses_on_Duty']
        ).fillna(0)
    
    if 'Doctors_on_Duty' in merged_df.columns:
        merged_df['Patients_per_Doctor'] = (
            merged_df['Patients_Count'] / merged_df['Doctors_on_Duty']
        ).fillna(0)

print("✓ Staffing ratios calculated")

# Display merged dataset summary
print("\n" + "=" * 50)
print("MERGED DATASET SUMMARY")
print("=" * 50)
print(f"Total records: {len(merged_df):,}")
print(f"Total columns: {len(merged_df.columns)}")
print(f"\nKey Statistics:")
print(merged_df[['Time_to_Doctor', 'Total_ER_Time', 'Seen_Within_15min']].describe())

## 5. Exploratory Data Analysis (EDA)

Comprehensive visual analysis of ER efficiency metrics.

### 5.1 Wait Time Analysis

In [ ]:
# Wait Time Breakdown by Stage
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

wait_stages = {
    'Wait to\nRegistration': merged_df['Wait_to_Registration'].mean(),
    'Registration\nDuration': merged_df['Registration_Duration'].mean(),
    'Wait to\nTriage': merged_df['Wait_to_Triage'].mean() - merged_df['Wait_to_Registration'].mean() - merged_df['Registration_Duration'].mean(),
    'Triage\nDuration': merged_df['Triage_Duration'].mean(),
    'Wait to\nDoctor': merged_df['Wait_to_Doctor'].mean(),
    'Treatment\nDuration': merged_df['Treatment_Duration'].mean()
}

# Filter out negative or invalid values
wait_stages = {k: max(0, v) for k, v in wait_stages.items()}

colors = ['#ff6b6b' if 'Wait' in k else '#4ecdc4' for k in wait_stages.keys()]
bars = ax.bar(wait_stages.keys(), wait_stages.values(), color=colors, alpha=0.8, edgecolor='black')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f} min',
            ha='center', va='bottom', fontweight='bold')

ax.set_ylabel('Average Time (minutes)', fontsize=12, fontweight='bold')
ax.set_title('ER Wait Time Breakdown by Stage\n(Red = Wait Time, Teal = Processing Time)', 
             fontsize=14, fontweight='bold', pad=20)
ax.grid(axis='y', alpha=0.3)
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('outputs/wait_time_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Wait time breakdown chart saved")

In [ ]:
# Time-to-Doctor Distribution with 15-min threshold
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# Filter extreme outliers for better visualization
time_to_doctor_filtered = merged_df[merged_df['Time_to_Doctor'] < 300]['Time_to_Doctor']

ax.hist(time_to_doctor_filtered, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
ax.axvline(x=15, color='red', linestyle='--', linewidth=2, label='15-minute Target')

# Calculate percentage meeting target
pct_within_15 = (merged_df['Seen_Within_15min'].sum() / len(merged_df)) * 100
ax.text(15, ax.get_ylim()[1] * 0.9, f'{pct_within_15:.1f}% within 15 min', 
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
        fontsize=11, fontweight='bold')

ax.set_xlabel('Time to Doctor (minutes)', fontsize=12, fontweight='bold')
ax.set_ylabel('Number of Patients', fontsize=12, fontweight='bold')
ax.set_title('Distribution of Time-to-Doctor (with 15-Minute Target)', 
             fontsize=14, fontweight='bold', pad=20)
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/time_to_doctor_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Time-to-doctor distribution saved")

In [ ]:
# Total ER Time Distribution
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# Convert to hours and filter outliers
total_er_hours = merged_df['Total_ER_Time'] / 60
total_er_hours_filtered = total_er_hours[total_er_hours < 12]

ax.hist(total_er_hours_filtered, bins=50, color='darkgreen', alpha=0.7, edgecolor='black')

mean_hours = total_er_hours.mean()
ax.axvline(x=mean_hours, color='red', linestyle='--', linewidth=2, 
           label=f'Mean: {mean_hours:.1f} hours')

ax.set_xlabel('Total ER Time (hours)', fontsize=12, fontweight='bold')
ax.set_ylabel('Number of Patients', fontsize=12, fontweight='bold')
ax.set_title('Distribution of Total ER Visit Duration', 
             fontsize=14, fontweight='bold', pad=20)
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.savefig('outputs/total_er_time_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Total ER time distribution saved")

### 5.2 Temporal Patterns

In [ ]:
# Create temporal patterns figure
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Triage Level Performance
if 'Triage_Level' in merged_df.columns:
    triage_performance = merged_df.groupby('Triage_Level').agg({
        'Seen_Within_15min': 'mean',
        'Visit_ID': 'count'
    }).reset_index()
    triage_performance.columns = ['Triage_Level', 'Pct_Within_15min', 'Count']
    triage_performance = triage_performance.sort_values('Triage_Level')
    
    bars = axes[0, 0].bar(triage_performance['Triage_Level'], 
                          triage_performance['Pct_Within_15min'] * 100,
                          color='coral', alpha=0.8, edgecolor='black')
    
    # Add percentage labels
    for bar in bars:
        height = bar.get_height()
        axes[0, 0].text(bar.get_x() + bar.get_width()/2., height,
                        f'{height:.1f}%',
                        ha='center', va='bottom', fontweight='bold')
    
    axes[0, 0].set_ylabel('% Seen Within 15 Minutes', fontsize=11, fontweight='bold')
    axes[0, 0].set_xlabel('Triage Level', fontsize=11, fontweight='bold')
    axes[0, 0].set_title('Triage Level Performance', fontsize=12, fontweight='bold')
    axes[0, 0].grid(axis='y', alpha=0.3)
    axes[0, 0].set_ylim(0, 100)

# 2. Patient Arrivals by Hour
hourly_arrivals = merged_df['Hour'].value_counts().sort_index()
axes[0, 1].plot(hourly_arrivals.index, hourly_arrivals.values, 
                marker='o', linewidth=2, markersize=8, color='darkblue')
axes[0, 1].fill_between(hourly_arrivals.index, hourly_arrivals.values, alpha=0.3)
axes[0, 1].set_ylabel('Number of Arrivals', fontsize=11, fontweight='bold')
axes[0, 1].set_xlabel('Hour of Day', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Patient Arrivals by Hour', fontsize=12, fontweight='bold')
axes[0, 1].grid(alpha=0.3)
axes[0, 1].set_xticks(range(0, 24, 2))

# 3. Average Time-to-Doctor by Shift
shift_avg = merged_df.groupby('Shift')['Time_to_Doctor'].mean().sort_values(ascending=False)
colors_shift = ['#ff6b6b' if x > shift_avg.mean() else '#51cf66' for x in shift_avg.values]
bars = axes[1, 0].barh(shift_avg.index, shift_avg.values, color=colors_shift, 
                       alpha=0.8, edgecolor='black')

for i, bar in enumerate(bars):
    width = bar.get_width()
    axes[1, 0].text(width, bar.get_y() + bar.get_height()/2.,
                    f'{width:.1f} min',
                    ha='left', va='center', fontweight='bold', fontsize=10)

axes[1, 0].set_xlabel('Average Time to Doctor (minutes)', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Average Time-to-Doctor by Shift', fontsize=12, fontweight='bold')
axes[1, 0].grid(axis='x', alpha=0.3)

# 4. Average Time-to-Doctor by Day of Week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_avg = merged_df.groupby('Day_of_Week')['Time_to_Doctor'].mean()
day_avg = day_avg.reindex(day_order)

colors_day = ['#ff6b6b' if x > day_avg.mean() else '#51cf66' for x in day_avg.values]
bars = axes[1, 1].bar(range(len(day_avg)), day_avg.values, color=colors_day, 
                      alpha=0.8, edgecolor='black')

for bar in bars:
    height = bar.get_height()
    axes[1, 1].text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.0f}',
                    ha='center', va='bottom', fontweight='bold')

axes[1, 1].set_xticks(range(len(day_avg)))
axes[1, 1].set_xticklabels([d[:3] for d in day_avg.index], rotation=45)
axes[1, 1].set_ylabel('Average Time to Doctor (minutes)', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Average Time-to-Doctor by Day of Week', fontsize=12, fontweight='bold')
axes[1, 1].grid(axis='y', alpha=0.3)

plt.suptitle('Temporal Patterns Analysis', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('outputs/temporal_patterns.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Temporal patterns analysis saved")

### 5.3 Staffing Analysis

In [ ]:
# Staffing Impact Analysis
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. Patients per Nurse vs Wait Time
if 'Patients_per_Nurse' in merged_df.columns:
    # Create bins for patients per nurse
    merged_df['Nurse_Ratio_Bin'] = pd.cut(merged_df['Patients_per_Nurse'], 
                                           bins=[0, 5, 10, 15, 20, 100],
                                           labels=['0-5', '5-10', '10-15', '15-20', '20+'])
    
    nurse_impact = merged_df.groupby('Nurse_Ratio_Bin')['Time_to_Doctor'].mean()
    
    axes[0].plot(range(len(nurse_impact)), nurse_impact.values, 
                marker='o', linewidth=3, markersize=10, color='purple')
    axes[0].set_xticks(range(len(nurse_impact)))
    axes[0].set_xticklabels(nurse_impact.index)
    axes[0].set_ylabel('Average Time to Doctor (minutes)', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Patients per Nurse', fontsize=12, fontweight='bold')
    axes[0].set_title('Impact of Nurse Staffing on Wait Times', fontsize=13, fontweight='bold')
    axes[0].grid(alpha=0.3)

# 2. Patients per Doctor vs Wait Time
if 'Patients_per_Doctor' in merged_df.columns:
    # Create bins for patients per doctor
    merged_df['Doctor_Ratio_Bin'] = pd.cut(merged_df['Patients_per_Doctor'], 
                                            bins=[0, 10, 20, 30, 40, 200],
                                            labels=['0-10', '10-20', '20-30', '30-40', '40+'])
    
    doctor_impact = merged_df.groupby('Doctor_Ratio_Bin')['Time_to_Doctor'].mean()
    
    axes[1].plot(range(len(doctor_impact)), doctor_impact.values, 
                marker='s', linewidth=3, markersize=10, color='orangered')
    axes[1].set_xticks(range(len(doctor_impact)))
    axes[1].set_xticklabels(doctor_impact.index)
    axes[1].set_ylabel('Average Time to Doctor (minutes)', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Patients per Doctor', fontsize=12, fontweight='bold')
    axes[1].set_title('Impact of Doctor Staffing on Wait Times', fontsize=13, fontweight='bold')
    axes[1].grid(alpha=0.3)

plt.suptitle('Staffing Impact Analysis', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('outputs/staffing_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Staffing analysis saved")

### 5.4 Demographics and Outcomes

In [ ]:
# Demographics and Outcomes Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Satisfaction vs Wait Time
if 'Patient_Satisfaction' in merged_df.columns:
    satisfaction_wait = merged_df.groupby('Patient_Satisfaction')['Time_to_Doctor'].mean().sort_index()
    
    colors_sat = plt.cm.RdYlGn(np.linspace(0, 1, len(satisfaction_wait)))
    bars = axes[0, 0].bar(satisfaction_wait.index.astype(str), satisfaction_wait.values, 
                          color=colors_sat, alpha=0.8, edgecolor='black')
    
    for bar in bars:
        height = bar.get_height()
        axes[0, 0].text(bar.get_x() + bar.get_width()/2., height,
                        f'{height:.1f}',
                        ha='center', va='bottom', fontweight='bold')
    
    axes[0, 0].set_xlabel('Patient Satisfaction Score', fontsize=11, fontweight='bold')
    axes[0, 0].set_ylabel('Average Time to Doctor (minutes)', fontsize=11, fontweight='bold')
    axes[0, 0].set_title('Patient Satisfaction vs Wait Time', fontsize=12, fontweight='bold')
    axes[0, 0].grid(axis='y', alpha=0.3)

# 2. Satisfaction by Age Group
if 'Age' in merged_df.columns and 'Patient_Satisfaction' in merged_df.columns:
    merged_df['Age_Group'] = pd.cut(merged_df['Age'], 
                                     bins=[0, 18, 35, 50, 65, 100],
                                     labels=['0-18', '19-35', '36-50', '51-65', '65+'])
    
    age_satisfaction = merged_df.groupby('Age_Group')['Patient_Satisfaction'].mean()
    
    bars = axes[0, 1].bar(age_satisfaction.index.astype(str), age_satisfaction.values, 
                          color='skyblue', alpha=0.8, edgecolor='black')
    
    for bar in bars:
        height = bar.get_height()
        axes[0, 1].text(bar.get_x() + bar.get_width()/2., height,
                        f'{height:.2f}',
                        ha='center', va='bottom', fontweight='bold')
    
    axes[0, 1].set_xlabel('Age Group', fontsize=11, fontweight='bold')
    axes[0, 1].set_ylabel('Average Satisfaction Score', fontsize=11, fontweight='bold')
    axes[0, 1].set_title('Average Satisfaction by Age Group', fontsize=12, fontweight='bold')
    axes[0, 1].grid(axis='y', alpha=0.3)
    axes[0, 1].set_ylim(0, 5)

# 3. Disposition Distribution (Pie Chart)
if 'Disposition' in merged_df.columns:
    disposition_counts = merged_df['Disposition'].value_counts()
    
    colors_pie = ['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3', '#a6d854']
    wedges, texts, autotexts = axes[1, 0].pie(disposition_counts.values, 
                                               labels=disposition_counts.index,
                                               autopct='%1.1f%%',
                                               colors=colors_pie[:len(disposition_counts)],
                                               startangle=90)
    
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
        autotext.set_fontsize(10)
    
    axes[1, 0].set_title('Patient Disposition Distribution', fontsize=12, fontweight='bold')

# 4. Wait Time by Disposition
if 'Disposition' in merged_df.columns:
    disposition_wait = merged_df.groupby('Disposition')['Time_to_Doctor'].mean().sort_values(ascending=False)
    
    bars = axes[1, 1].barh(disposition_wait.index, disposition_wait.values, 
                           color='lightcoral', alpha=0.8, edgecolor='black')
    
    for bar in bars:
        width = bar.get_width()
        axes[1, 1].text(width, bar.get_y() + bar.get_height()/2.,
                        f'{width:.1f} min',
                        ha='left', va='center', fontweight='bold', fontsize=10)
    
    axes[1, 1].set_xlabel('Average Time to Doctor (minutes)', fontsize=11, fontweight='bold')
    axes[1, 1].set_title('Average Wait Time by Disposition', fontsize=12, fontweight='bold')
    axes[1, 1].grid(axis='x', alpha=0.3)

plt.suptitle('Demographics and Outcomes Analysis', fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('outputs/demographics_outcomes.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Demographics and outcomes analysis saved")

### 5.5 Correlation Analysis

In [ ]:
# Correlation Matrix
numeric_cols = merged_df.select_dtypes(include=[np.number]).columns.tolist()

# Select key features for correlation
key_features = ['Time_to_Doctor', 'Total_ER_Time', 'Wait_to_Doctor', 'Treatment_Duration',
                'Triage_Duration', 'Patients_per_Nurse', 'Patients_per_Doctor', 
                'Patient_Satisfaction', 'Age', 'Is_Weekend', 'Hour']

# Filter to only existing columns
key_features = [col for col in key_features if col in numeric_cols]

correlation_matrix = merged_df[key_features].corr()

# Create correlation heatmap
fig, ax = plt.subplots(figsize=(14, 10))

sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8},
            vmin=-1, vmax=1, ax=ax)

ax.set_title('Correlation Matrix of Key ER Efficiency Metrics', 
             fontsize=14, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('outputs/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Print top correlations with wait time
print("\nTop Correlations with Time_to_Doctor:")
print("=" * 50)
time_to_doctor_corr = correlation_matrix['Time_to_Doctor'].sort_values(ascending=False)
print(time_to_doctor_corr[time_to_doctor_corr.index != 'Time_to_Doctor'].head(10))

if 'Patient_Satisfaction' in correlation_matrix.columns:
    print("\nTop Correlations with Patient_Satisfaction:")
    print("=" * 50)
    satisfaction_corr = correlation_matrix['Patient_Satisfaction'].sort_values(ascending=False)
    print(satisfaction_corr[satisfaction_corr.index != 'Patient_Satisfaction'].head(10))

print("\n✓ Correlation analysis saved")